# Code for doing simulations
Here I upload all the necessary stuff

In [ ]:
import numpy as np
from simulated_dipole import *
import pandas as pd


# IN CASE OF AixNat
# input_path = '/mnt/c/Users/paolo/Desktop/LAVORO/data_files/aixnet_sib_all_data.npz'
# data_all = np.load(input_path, allow_pickle=True)
# ddd = {key: data_all[key] for key in data_all.files}


## IN CASE OF KAne
input_kane = '../berenika_dipole/dateset/KAne_EPOS_sims.csv'

kf = pd.read_csv(input_kane)
ddd = {}
ddd['dnn_xmax'] = np.array(kf['Xmax'])
ddd['zenith'] = np.array(np.pi/2 - kf['Zenith'])
ddd['energy'] = np.array(kf['Energy']/1e18)
ddd['primary'] = np.array(kf['Primary'])
ddd['mass'] = np.empty_like(ddd['energy'])
ddd['mass'][np.where(ddd['primary']==0)] = 1
ddd['mass'][np.where(ddd['primary']==1)] = 4
ddd['mass'][np.where(ddd['primary']==2)] = 16
ddd['mass'][np.where(ddd['primary']==3)] = 56



print(ddd.keys())
print('Number of simulated events:', len(ddd['energy']))

n_samples = 100
np.random.seed(90)
seed_1 = np.random.randint(1000, size=n_samples)
seed_2 = np.random.randint(1000, size=n_samples)
seed_3 = np.random.randint(1000, size=n_samples)

dict_keys(['dnn_xmax', 'zenith', 'energy', 'primary', 'mass'])
Number of simulated events: 808809


## Data production cycle

In [2]:
## classes I need (AixNet)
# mf = mass_fractions()
# es = energy_spectrum()
# smd = SMD_method()

## IN CASE OF KAne
mf = mass_fractions(model='EPOS')
es = energy_spectrum()
smd = SMD_method(model='EPOS')

# final results dictionary
final_results = {'iter': [], 'emin': [], 'light': [], 'heavy': [], 'frac_l': [], 'frac_h': []}

for k in range(n_samples):

    print('Iteration:', k+1, ' of ', n_samples)
    # data_mask = flatten_distr(ddd['energy'], seed=seed_1[k]) # only for AixNet
    # dd = dict_cutter(ddd, data_mask)

    dd = ddd # only for KAne
    print('After the first cut:', len(dd['energy']))

    # extract mass fractions
    mf.seed = seed_2[k]
    mass_mask = mf.extract_all_fractions(np.log10(dd['energy']*1e18), dd['mass'])
    dd = dict_cutter(dd, mass_mask)
    print('After mass fractions:', len(dd['energy']))

    # extract spectrum
    es.seed = seed_3[k]

    temp_dict = [{} for i in range(4)]

    for i in range(len(energies[:-1])):
        enne = energies[i]/1e18
        enneplus = energies[i+1]/1e18
        energy_mask = (dd['energy']>=enne)&(dd['energy']<enneplus)
        temp_dict[i] = dict_cutter(dd, energy_mask)
        print(temp_dict[i].keys())
        spectrum_mask = es.spectrum_fraction(temp_dict[i]['energy']*1e18)
        temp_dict[i] = dict_cutter(temp_dict[i], spectrum_mask)

    dd = dict_paster(temp_dict)
    print('After energy spectrum:', len(dd['energy']))

    # define charge
    dd['charge'] = np.empty_like(dd['mass'])
    for mass in names:
        dd['charge'][np.where(dd['mass']==names[mass])]=charges[mass]
    
    generate_xmax19(dd, 'dnn_xmax', 'energy')
    
    # estimate best quantiles
    thres_values = np.linspace(np.min(dd['xmax19']), np.max(dd['xmax19']), 27)
    res = { 'emin': [], 
            'light': [], 
            'heavy': [], 
            'N_light': [],
            'N_heavy': [],
            'N_tot': [],
            'SMD': [],
            }
    
    for ee in range(len(energies)-1):
        e_min = energies[ee]/1e18
        e_max =energies[ee+1]/1e18


        # extract energy range
        e_mask = (dd['energy']<e_max)&(dd['energy']>=e_min)
        data = dict_cutter(dd, e_mask)

        for i in range(1, len(thres_values) - 2):
            thres_h = thres_values[i]

            for j in range(i + 1, len(thres_values) - 1):

                thres_l = thres_values[j]
                mask_h = (data['xmax19']<thres_h)
                mask_l = (data['xmax19']>thres_l)
                light_frac = dict_cutter(data, mask_l)
                heavy_frac = dict_cutter(data, mask_h)

                value = smd.quantify_SMD(light_frac['energy'], light_frac['charge'], heavy_frac['energy'], heavy_frac['charge'], d_max=1)
                A_l = np.sum(np.log(light_frac['mass']))
                A_h = np.sum(np.log(heavy_frac['mass']))

                res['emin'].append(e_min)
                res['light'].append(thres_l)
                res['heavy'].append(thres_h)
                res['SMD'].append(value)
                res['N_light'].append(len(light_frac['xmax19']))
                res['N_heavy'].append(len(heavy_frac['xmax19']))
                res['N_tot'].append(len(data['xmax19']))

    for key, value in res.items():
        res[key] = np.array(value)

    # format final results
    for l in range(len(energies[:-1])):
        best_pos = np.where((res['SMD']==np.max(res['SMD'][res['emin']==energies[l]/1e18]))&(res['emin']==energies[l]/1e18))
        print(best_pos)
        frac_l = res['N_light'][best_pos]/res['N_tot'][best_pos]
        frac_h = res['N_heavy'][best_pos]/res['N_tot'][best_pos]

        final_results['iter'].append(k)
        final_results['emin'].append(energies[l]/1e18)
        final_results['light'].append(res['light'][best_pos])
        final_results['heavy'].append(res['heavy'][best_pos])
        final_results['frac_l'].append(frac_l)
        final_results['frac_h'].append(frac_h)


for key, value in final_results.items():
    final_results[key] = np.array(value)
display(final_results)

Iteration: 1  of  1000
After the first cut: 808809
After mass fractions: 198035
dict_keys(['dnn_xmax', 'zenith', 'energy', 'primary', 'mass'])
dict_keys(['dnn_xmax', 'zenith', 'energy', 'primary', 'mass'])
dict_keys(['dnn_xmax', 'zenith', 'energy', 'primary', 'mass'])
dict_keys(['dnn_xmax', 'zenith', 'energy', 'primary', 'mass'])
After energy spectrum: 49771
(array([93]),)
(array([392]),)
(array([692]),)
(array([972]),)
Iteration: 2  of  1000
After the first cut: 808809
After mass fractions: 198507
dict_keys(['dnn_xmax', 'zenith', 'energy', 'primary', 'mass'])
dict_keys(['dnn_xmax', 'zenith', 'energy', 'primary', 'mass'])
dict_keys(['dnn_xmax', 'zenith', 'energy', 'primary', 'mass'])
dict_keys(['dnn_xmax', 'zenith', 'energy', 'primary', 'mass'])
After energy spectrum: 49666
(array([91]),)
(array([371]),)
(array([671]),)
(array([970]),)
Iteration: 3  of  1000
After the first cut: 808809
After mass fractions: 199017
dict_keys(['dnn_xmax', 'zenith', 'energy', 'primary', 'mass'])
dict_keys

KeyboardInterrupt: 

In [ ]:
display(final_results)
print(len(final_results['light']))

{'iter': array([  0,   0,   0, ..., 999, 999, 999]),
 'emin': array([ 4.,  8., 16., ...,  8., 16., 32.]),
 'light': [array([818.8524], dtype=float32),
  array([788.943], dtype=float32),
  array([773.9883], dtype=float32),
  array([773.9883], dtype=float32),
  array([807.68896], dtype=float32),
  array([778.1587], dtype=float32),
  array([778.1587], dtype=float32),
  array([763.39355], dtype=float32),
  array([782.388], dtype=float32),
  array([817.81854], dtype=float32),
  array([782.388], dtype=float32),
  array([764.6728], dtype=float32),
  array([796.9452], dtype=float32),
  array([796.9452], dtype=float32),
  array([796.9452], dtype=float32),
  array([766.4459], dtype=float32),
  array([815.10065], dtype=float32),
  array([786.2057], dtype=float32),
  array([786.2057], dtype=float32),
  array([771.7582], dtype=float32),
  array([794.6357], dtype=float32),
  array([794.6357], dtype=float32),
  array([777.44446], dtype=float32),
  array([777.44446], dtype=float32),
  array([799.48157

4000


In [ ]:
np.save('results.npy', final_results)
df = pd.DataFrame(final_results)
df.to_csv("final_results_KAne.csv", index=False)